In [1]:
import numpy as np
import pysmiles as pys
import networkx as nx
import urllib as ulib
import scipy as scp

In [2]:
# Defining functions
def smile_convert(chemname):
    '''
    Function uses cactus.nci.nih.gov website to fetch the smile string corresponding
    to the input molecule
    Inputs can be of several forms: 
    IUPAC name
    CAS number

    Other input forms may work as the website is updated but these are best for
    getting the correct result easily
    
    This will also accept molecules containing non-carbon atoms, meaning the 
    program could be expanded to drop the assumption that all atoms are carbon
    '''
    try:
        url = 'https://cactus.nci.nih.gov/chemical/structure/' + ulib.parse.quote(chemname) + '/smiles'
        smilestring = ulib.request.urlopen(url).read().decode('utf8')
        return smilestring
    except:
        print('Failed to convert to smile string')

def get_evals(matrix):
    '''
    Input in the form of square matrix is required 

    eigh function saves computation as only considers upper triangular part of
    matrix, so symmetric matrix input needed also

    Function also returns eigenvalues in ascending order + groups degenerate 
    eigenvalues, meaning only adjacent values need to be checked for degeneracy

    Return eigenvalues and eigenvectors of the matrix
    '''
    evals, evec = np.linalg.eigh(matrix)
    return evals

In [ ]:
# Define dictionary for selecting platonic solid easily
platonics = dict({
    '4':nx.tetrahedral_graph(),
    '6':nx.octahedral_graph(),
    '8':nx.cubical_graph(),
    '12':nx.dodecahedral_graph(),
    '20':nx.icosahedral_graph()
})

# Input of molecule and conversion to a networkx graph object
molec_cat = input('Select number to calculate for: 1.Molecule by name,\
2.Platonic Solid, 3.Molecule by SMILE string:  ')

if molec_cat == '1':
    molec_in = input('Input molecule IUPAC name or CAS number:  ')
    molec_smile = smile_convert(molec_in)
    molec_graph = pys.read_smiles(molec_smile,explicit_hydrogen=False,zero_order_bonds=True)
elif molec_cat == '2':
    n = input('Input number of vetrices in solid:  ')
    molec_graph = platonics[n]
elif molec_cat == '3':
    molec_smile = input('Input molecule smile string:  ')
    molec_graph = pys.read_smiles(molec_smile,explicit_hydrogen=False,zero_order_bonds=True)

# Extract adjacency matrix from networkx graph object
adj_matrix = nx.adjacency_matrix(molec_graph)
adj_matrix = adj_matrix.toarray()

C:\Users\domin\AppData\Local\Temp\ipykernel_16704\3907054218.py:25: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(molec_graph)


In [4]:
# Create and solve Huckel matrix
# Can switch commented lines below to have alpha and beta as inputs or preset

# alpha = input('Alpha value to use')
# beta = input('Beta value to use')
alpha = 1
beta = 2

# Convert adjacency matrix to huckel matrix to solve
huckel_matrix = adj_matrix*beta + np.identity(len(adj_matrix))*alpha

In [5]:
# Calculate eigenvalues
energy = get_evals(huckel_matrix)

In [78]:
# Define result array to write to, inf means first value always appended
eig_out = np.array(([np.Inf,np.Inf]),dtype=object)
eig_out.shape = (1,2)                      # defining shape stops index error

i = 0

# Check each for degeneracy within a factor of 1% beta
for eig in energy:
    if abs(eig_out[i,0] - eig) <= 0.01*beta:
        eig_out[i,1] += 1
    else:
        eig_join = np.array([np.round(eig,4),1])    # Round energies to 4dp
        eig_out = np.vstack((eig_out,eig_join))     # may need to change later
        i += 1

eig_out[0,0] = 'Energy'
eig_out[0,1] = 'Degeneracy'

In [79]:
print(eig_out)

[['Energy' 'Degeneracy']
 [-3.0 1.0]
 [-1.0 2.0]
 [3.0 2.0]
 [5.0 1.0]]
